# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [115]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from env.api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [116]:
# Store filepath in a variable
input_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(input_file)
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,haines junction,99,CA,1579743204,97,60.75,-137.51,6.76,4.14
1,ushuaia,75,AR,1579743204,71,-54.80,-68.30,50.00,9.17
2,constitucion,59,CL,1579743164,94,-35.33,-72.42,56.61,5.23
3,vaini,40,TO,1579743101,79,-21.20,-175.20,84.20,13.87
4,sidi ali,0,DZ,1579743204,78,36.10,0.42,47.03,5.19
...,...,...,...,...,...,...,...,...,...
537,portland,90,US,1579743739,87,45.52,-122.68,54.00,12.75
538,inhambane,68,MZ,1579743799,65,-23.86,35.38,76.35,7.09
539,san carlos de bariloche,20,AR,1579743859,40,-41.15,-71.31,69.80,10.29
540,praia,95,CV,1579743595,83,14.92,-23.51,69.80,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [117]:
# Access maps with API key
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
rating = cities_df["Humidity"].astype(float)

In [119]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [120]:
df = cities_df.copy()
df = df.loc[(df['Max Temp'] < 80) & (df['Max Temp'] > 70) & (df['Wind Speed'] < 20) & (df['Cloudiness'] < 10)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [121]:
hotel_df = df.copy()
hotel_df['Hotel Name'] = ""

In [122]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [123]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']

    # set up params dict
    params = {
    "location": f"{lat},{lng}",
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 17: kapaa.
Closest hotel is Courtyard by Marriott Kaua'i at Coconut Beach.
------------
Retrieving Results for Index 56: okakarara.
Missing field/result... skipping.
------------
Retrieving Results for Index 83: carnarvon.
Closest hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 142: vao.
Closest hotel is Hotel Kou-Bugny.
------------
Retrieving Results for Index 216: tandil.
Closest hotel is Ruphay.
------------
Retrieving Results for Index 219: machali.
Closest hotel is OP Hotels.
------------
Retrieving Results for Index 380: goya.
Closest hotel is Condado Hotel Casino.
------------
Retrieving Results for Index 392: sefton.
Closest hotel is Terrace Lodge.
------------
Retrieving Results for Index 414: rio grande.
Closest hotel is Laghetto Viverone Rio Grande.
------------
Retrieving Results for Index 451: kontagora.
Closest hotel is City Gate Inn.
------------
Retrieving Results for Index 466: poum.
Missing field/result... skippi

In [127]:
#df = hotel_df.dropna(how='any', inplace = True)
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [128]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [126]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))